In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


## Project Day

### Importing the dataset

In [2]:
df=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding='latin-1')
#dropping unrelated columns
df=df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
#renaming columns
df=df.rename(columns={
     'v1':'label',
    'v2':'message'
}
)
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
#number of rows and columns
df.shape

(5572, 2)

### Data cleaning and preprocessing

In [4]:
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [6]:
ps=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [7]:
corpus=[]
for i in range(len(df)):
    review= re.sub('[^a-zA-Z]',' ',df['message'][i])
    review= review.lower()
    review= review.split()
    review= [ps.stem(word) for word in review if word not in set(stopwords.words("english"))]
    review= ' '.join(review)
    corpus.append(review)
    

### Creating bags of words model

X for message 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500,binary=True,ngram_range=(2,2))
X= cv.fit_transform(corpus).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

y for labels

In [9]:
y= pd.get_dummies(df['label'])
y=y.iloc[:,1].values

### Train,Test Split

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=0)

In [12]:
X_train,y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([False, False, False, ..., False, False, False]))

### Model building

In [13]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model= MultinomialNB().fit(X_train,y_train)

### Prediction

In [14]:
y_pred= spam_detect_model.predict(X_test)

### Model evaluation

In [15]:
from sklearn.metrics import accuracy_score,classification_report

In [16]:
score= accuracy_score(y_test,y_pred)
print(score)

0.9632286995515695


In [17]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       1.00      0.96      0.98       988
        True       0.76      0.99      0.86       127

    accuracy                           0.96      1115
   macro avg       0.88      0.98      0.92      1115
weighted avg       0.97      0.96      0.97      1115



### TF/IDF model

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X=tv.fit_transform(corpus).toarray()

### Train,Test,Split

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=0)

In [21]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model= MultinomialNB().fit(X_train,y_train)

In [22]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score,classification_report

In [24]:
score= accuracy_score(y_test,y_pred)
print(score)

0.9704035874439462


In [25]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       1.00      0.97      0.98       980
        True       0.81      0.99      0.89       135

    accuracy                           0.97      1115
   macro avg       0.90      0.98      0.94      1115
weighted avg       0.98      0.97      0.97      1115



### RandomForest

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [27]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [28]:
y_pred=classifier.predict(X_test)

In [29]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9829596412556054
              precision    recall  f1-score   support

       False       1.00      0.98      0.99       968
        True       0.89      1.00      0.94       147

    accuracy                           0.98      1115
   macro avg       0.94      0.99      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [30]:
!pip install joblib

In [31]:
import joblib

In [32]:
joblib.dump(classifier,"clasi_model.pkl")

['clasi_model.pkl']

In [33]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.9 MB/s eta 0:00:00
